# Tensor and Add Operation

ttnn.Tensor is the central type of ttnn.

It is similar to torch.Tensor in the sense that it represents multi-dimensional matrix containing elements of a single data type.

The are a few key differences:

- ttnn.Tensor can be stored in the SRAM or DRAM of Tenstorrent devices
- ttnn.Tensor doesn't have a concept of the strides, however it has a concept of row-major and tile layout
- ttnn.Tensor has support for data types not supported by torch such as `bfp8` for example
- ttnn.Tensor's shape stores the padding added to the tensor due to TILE_LAYOUT

## Creating a tensor

The recommended way to create a tensor is by using torch create function and then simply calling `ttnn.from_torch`. So, let's import both `torch` and `ttnn`

In [1]:
import torch
import ttnn

2025-06-19 15:14:51.755 | DEBUG    | ttnn:<module>:83 - Initial ttnn.CONFIG:
Config{cache_path=/home/rsong/.cache/ttnn,model_cache_path=/home/rsong/.cache/ttnn/models,tmp_dir=/tmp/ttnn,enable_model_cache=false,enable_fast_runtime_mode=true,throw_exception_on_fallback=false,enable_logging=false,enable_graph_report=false,enable_detailed_buffer_report=false,enable_detailed_tensor_report=false,enable_comparison_mode=false,comparison_mode_should_raise_exception=false,comparison_mode_pcc=0.9999,root_report_path=generated/ttnn/reports,report_name=std::nullopt,std::nullopt}


In [2]:
import os


And now let's create a torch Tensor and convert it to ttnn Tensor

In [3]:
torch_tensor = torch.rand(3, 4)
ttnn_tensor = ttnn.from_torch(torch_tensor)

print(f"shape: {ttnn_tensor.shape}")
print(f"layout: {ttnn_tensor.layout}")
print(f"dtype: {ttnn_tensor.dtype}")

shape: Shape([3, 4])
layout: Layout.ROW_MAJOR
dtype: DataType.FLOAT32


As expected we get a tensor of shape [3, 4] in row-major layout with a data type of float32.

## Host Storage: Borrowed vs Owned

In this particular case, ttnn Tensor will borrow the data of the torch Tensor because ttnn Tensor is in row-major layout, torch tensor is contiguous and their data type matches.

Let's print the current ttnn tensor, set element of torch tensor to 1234 and print the ttnn Tensor again to see borrowed storage in action

In [4]:
print(f"Original values:\n{ttnn_tensor}")
torch_tensor[:] = 1234
print(f"New values are all going to be 1234:\n{ttnn_tensor}")

Original values:
ttnn.Tensor([[ 0.65476,  0.08116,  0.99986,  0.66685],
             [ 0.12592,  0.63512,  0.27911,  0.77591],
             [ 0.53017,  0.43541,  0.43046,  0.18515]], shape=Shape([3, 4]), dtype=DataType::FLOAT32, layout=Layout::ROW_MAJOR)
New values are all going to be 1234:
ttnn.Tensor([[1234.00000, 1234.00000, 1234.00000, 1234.00000],
             [1234.00000, 1234.00000, 1234.00000, 1234.00000],
             [1234.00000, 1234.00000, 1234.00000, 1234.00000]], shape=Shape([3, 4]), dtype=DataType::FLOAT32, layout=Layout::ROW_MAJOR)


We try our best to use borrowed storage but if the torch data type is not supported in ttnn, then we don't have a choice but to automatically pick a different data type and copy data

In [5]:
torch_tensor = torch.rand(3, 4).to(torch.float16)
ttnn_tensor = ttnn.from_torch(torch_tensor)
print("torch_tensor.dtype:", torch_tensor.dtype)
print("ttnn_tensor.dtype:", ttnn_tensor.dtype)

print(f"Original values:\n{ttnn_tensor}")
torch_tensor[0, 0] = 1234
print(f"Original values again because the tensor doesn't use borrowed storage:\n{ttnn_tensor}")

torch_tensor.dtype: torch.float16
ttnn_tensor.dtype: DataType.BFLOAT16
Original values:
ttnn.Tensor([[ 0.73047,  0.48828,  0.65234,  0.76562],
             [ 0.42188,  0.42578,  0.85547,  0.39258],
             [ 0.08447,  0.71875,  0.04395,  0.02368]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)
Original values again because the tensor doesn't use borrowed storage:
ttnn.Tensor([[ 0.73047,  0.48828,  0.65234,  0.76562],
             [ 0.42188,  0.42578,  0.85547,  0.39258],
             [ 0.08447,  0.71875,  0.04395,  0.02368]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


## Data Type

The data type of the ttnn tensor can be controlled explicitly when conversion from torch.

In [6]:
torch_tensor = torch.rand(3, 4).to(torch.float32)
ttnn_tensor = ttnn.from_torch(torch_tensor, dtype=ttnn.bfloat16)
print(f"torch_tensor.dtype: {torch_tensor.dtype}")
print(f"ttnn_tensor.dtype: {ttnn_tensor.dtype}")

torch_tensor.dtype: torch.float32
ttnn_tensor.dtype: DataType.BFLOAT16


## Layout

Tenstorrent hardware is most efficiently utilized when running tensors using [tile layout](https://tenstorrent.github.io/ttnn/latest/ttnn/tensor.html#layout).
The current tile size is hard-coded to [32, 32]. It was determined to be the optimal size for a tile given the compute, memory and data transfer constraints.


ttnn provides easy and intuitive way to convert from row-major layout to tile layout and back.

In [7]:
torch_tensor = torch.rand(3, 4).to(torch.float16)
ttnn_tensor = ttnn.from_torch(torch_tensor)
print(f"Tensor in row-major layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")
ttnn_tensor = ttnn.to_layout(ttnn_tensor, ttnn.TILE_LAYOUT)
print(f"Tensor in tile layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")
ttnn_tensor = ttnn.to_layout(ttnn_tensor, ttnn.ROW_MAJOR_LAYOUT)
print(f"Tensor back in row-major layout:\nShape {ttnn_tensor.shape}\nLayout: {ttnn_tensor.layout}\n{ttnn_tensor}")

Tensor in row-major layout:
Shape Shape([3, 4])
Layout: Layout.ROW_MAJOR
ttnn.Tensor([[ 0.58984,  0.71094,  0.55078,  0.81250],
             [ 0.09570,  0.06030,  0.42969,  0.31641],
             [ 0.30078,  0.06152,  0.91016,  0.09912]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)
Tensor in tile layout:
Shape Shape([3, 4])
Layout: Layout.TILE
ttnn.Tensor([[ 0.58984,  0.71094,  0.55078,  0.81250],
             [ 0.09570,  0.06030,  0.42969,  0.31641],
             [ 0.30078,  0.06152,  0.91016,  0.09912]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::TILE)
Tensor back in row-major layout:
Shape Shape([3, 4])
Layout: Layout.ROW_MAJOR
ttnn.Tensor([[ 0.58984,  0.71094,  0.55078,  0.81250],
             [ 0.09570,  0.06030,  0.42969,  0.31641],
             [ 0.30078,  0.06152,  0.91016,  0.09912]], shape=Shape([3, 4]), dtype=DataType::BFLOAT16, layout=Layout::ROW_MAJOR)


Note that padding is automatically inserted to put the tensor into tile layout and it automatically removed after the tensor is converted back to row-major layout

The conversion to tile layout can be done when caling `ttnn.from_torch`

In [8]:
torch_tensor = torch.rand(3, 4).to(torch.float16)
ttnn_tensor = ttnn.from_torch(torch_tensor, layout=ttnn.TILE_LAYOUT)
print(f"Tensor in tile layout:\nShape {ttnn_tensor.shape}; Layout: {ttnn_tensor.layout}")
torch_tensor = ttnn.to_torch(ttnn_tensor)
print(f"Tensor in row-major layout:\nShape {torch_tensor.shape}; Layout: {torch_tensor.layout}")

Tensor in row-major layout:
Shape Shape([3, 4]); Layout: Layout.TILE
Tensor in row-major layout:
Shape torch.Size([3, 4]); Layout: torch.strided


Note that `ttnn.to_torch` will always convert to row-major layout

## Device storage

Finally, in order to actually utilize the tensor, we need to put it on the device. So, that we can run `ttnn` operations on it

## Open the device

Use `ttnn.open` to get a handle to the device

In [13]:
device_id = 0
device = ttnn.open_device(device_id=device_id)

2025-06-17 19:38:48.892 | info     |   SiliconDriver | Opened PCI device 7; KMD version: 1.34.0, IOMMU: disabled (pci_device.cpp:191)
2025-06-17 19:38:48.894 | info     |   SiliconDriver | Opened PCI device 7; KMD version: 1.34.0, IOMMU: disabled (pci_device.cpp:191)
2025-06-17 19:38:48.923 | info     |          Device | Opening user mode device driver (tt_cluster.cpp:177)
2025-06-17 19:38:48.923 | info     |   SiliconDriver | Opened PCI device 7; KMD version: 1.34.0, IOMMU: disabled (pci_device.cpp:191)
2025-06-17 19:38:48.924 | info     |   SiliconDriver | Opened PCI device 7; KMD version: 1.34.0, IOMMU: disabled (pci_device.cpp:191)
2025-06-17 19:38:48.929 | info     |   SiliconDriver | Opened PCI device 7; KMD version: 1.34.0, IOMMU: disabled (pci_device.cpp:191)
2025-06-17 19:38:48.930 | info     |   SiliconDriver | Opened PCI device 7; KMD version: 1.34.0, IOMMU: disabled (pci_device.cpp:191)
2025-06-17 19:38:48.935 | info     |   SiliconDriver | Harvesting mask for chip 0 is 0x0

## Initialize tensors a and b with random values using torch

To create a tensor that can be used by a `ttnn` operation:
1. Create a tensor using torch
2. Use `ttnn.from_torch` to convert the tensor from `torch.Tensor` to `ttnn.Tensor`, change the layout to `ttnn.TILE_LAYOUT` and put the tensor on the `device`

In [14]:
torch.manual_seed(0)

torch_input_tensor_a = torch.rand((32, 32), dtype=torch.bfloat16)
torch_input_tensor_b = torch.rand((32, 32), dtype=torch.bfloat16)

input_tensor_a = ttnn.from_torch(torch_input_tensor_a, layout=ttnn.TILE_LAYOUT, device=device)
input_tensor_b = ttnn.from_torch(torch_input_tensor_b, layout=ttnn.TILE_LAYOUT, device=device)

## Add tensor a and b

`ttnn` supports operator overloading, therefore operator `+` can be used instead of `torch.add`

In [23]:
output_tensor = input_tensor_a + input_tensor_b

## Inspect the output tensor of the add in ttnn

As can be seen the tensor of the same shape, layout and dtype is produced

In [24]:
print(f"shape: {output_tensor.shape}")
print(f"dtype: {output_tensor.dtype}")
print(f"layout: {output_tensor.layout}")

shape: Shape([32, 32])
dtype: DataType.BFLOAT16
layout: Layout.TILE


In general we expect layout and dtype to stay the same when running most operations unless explicit arguments to modify them are passed in. However, there are obvious exceptions like an embedding operation that takes in `ttnn.uint32` and produces `ttnn.bfloat16`

## Convert to torch and inspect the attributes of the torch tensor

When converting the tensor to torch, `ttnn.to_torch` will move the tensor from the device, convert to tile layout and figure out the best data type to use on the torch side

In [25]:
output_tensor = ttnn.to_torch(output_tensor)
print(f"shape: {output_tensor.shape}")
print(f"dtype: {output_tensor.dtype}")

shape: torch.Size([32, 32])
dtype: torch.bfloat16


## Close the device

Close the handle the device. This is a very important step as the device can hang currently if not closed properly

In [26]:
ttnn.close_device(device)

2025-06-17 19:40:48.519 | info     |           Metal | Closing mesh device 1 (mesh_device.cpp:486)
2025-06-17 19:40:48.519 | info     |           Metal | Closing mesh device 0 (mesh_device.cpp:486)
2025-06-17 19:40:48.520 | info     |           Metal | Closing device 0 (device.cpp:1097)
2025-06-17 19:40:48.520 | info     |           Metal | Disabling and clearing program cache on device 0 (device.cpp:1442)
